In [16]:
import requests
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import numpy as np
import plotly.express as px

In [17]:
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tps00098?format=JSON&time=2011&time=2012&time=2013&time=2014&time=2015&time=2016&time=2017&time=2018&time=2019&time=2020&time=2021&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&geo=IS&geo=NO&geo=CH&geo=BA&geo=RS&geo=TR&unit=PC_GDP&spdeps=TOTAL&lang=en"
response = requests.get(url)
data = response.json()
dimensions = data['dimension']
values = data['value']


rows = []


dim_names = list(dimensions.keys())


for index, value in values.items():
    row = {}
        

    idx = int(index)
        

    temp_idx = idx
    for dim_name in reversed(dim_names):
        dim_size = len(dimensions[dim_name]['category']['index'])
        dim_idx = temp_idx % dim_size
        temp_idx = temp_idx // dim_size
            
    
        dim_keys = list(dimensions[dim_name]['category']['index'].keys())
        category_key = dim_keys[dim_idx]
            
        
        if 'label' in dimensions[dim_name]['category']:
            row[dim_name] = dimensions[dim_name]['category']['label'].get(category_key, category_key)
        else:
            row[dim_name] = category_key
        
    
        row['value'] = value
        rows.append(row)


    df = pd.DataFrame(rows)
    df_clean = df.drop(["spdeps","freq","unit"], axis =1 )
    df_clean = df_clean[df_clean['geo'] != 'Türkiye']  

In [18]:
display(df_clean)
print(df_clean.time.tolist())

,time,value,geo
0,2011,28.96,Austria
1,2011,28.96,Austria
2,2011,28.96,Austria
3,2011,28.96,Austria
4,2011,28.96,Austria
...,...,...,...
1905,2022,18.18,Slovakia
1906,2022,18.18,Slovakia
1907,2022,18.18,Slovakia
1908,2022,18.18,Slovakia


['2011', '2011', '2011', '2011', '2011', '2012', '2012', '2012', '2012', '2012', '2013', '2013', '2013', '2013', '2013', '2014', '2014', '2014', '2014', '2014', '2015', '2015', '2015', '2015', '2015', '2016', '2016', '2016', '2016', '2016', '2017', '2017', '2017', '2017', '2017', '2018', '2018', '2018', '2018', '2018', '2019', '2019', '2019', '2019', '2019', '2020', '2020', '2020', '2020', '2020', '2021', '2021', '2021', '2021', '2021', '2022', '2022', '2022', '2022', '2022', '2013', '2013', '2013', '2013', '2013', '2014', '2014', '2014', '2014', '2014', '2015', '2015', '2015', '2015', '2015', '2016', '2016', '2016', '2016', '2016', '2017', '2017', '2017', '2017', '2017', '2018', '2018', '2018', '2018', '2018', '2019', '2019', '2019', '2019', '2019', '2020', '2020', '2020', '2020', '2020', '2021', '2021', '2021', '2021', '2021', '2022', '2022', '2022', '2022', '2022', '2011', '2011', '2011', '2011', '2011', '2012', '2012', '2012', '2012', '2012', '2013', '2013', '2013', '2013', '2013',

In [19]:
df_clean = df_clean.drop_duplicates()
display(df_clean)


,time,value,geo
0,2011,28.96,Austria
5,2012,29.29,Austria
10,2013,29.83,Austria
15,2014,30.00,Austria
20,2015,29.97,Austria
...,...,...,...
1885,2018,17.82,Slovakia
1890,2019,17.81,Slovakia
1895,2020,19.40,Slovakia
1900,2021,19.03,Slovakia


In [36]:
df_clean = df_clean.sort_values(by=["time", "geo"])
display(df_clean)


,time,value,geo,lag_1,lag_2,lag_3,lag_4,lag_5
0,2011,28.96,Austria,NaN,NaN,NaN,NaN,NaN
12,2011,29.71,Belgium,NaN,NaN,NaN,NaN,NaN
34,2011,16.43,Bulgaria,NaN,NaN,NaN,NaN,NaN
46,2011,20.58,Croatia,NaN,NaN,NaN,NaN,NaN
58,2011,20.03,Cyprus,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
333,2022,18.18,Slovakia,NaN,NaN,NaN,NaN,NaN
345,2022,24.25,Slovenia,NaN,NaN,NaN,NaN,NaN
357,2022,25.87,Spain,NaN,NaN,NaN,NaN,NaN
369,2022,27.23,Sweden,NaN,NaN,NaN,NaN,NaN


In [38]:
y_vals = df_clean[(df_clean["time"] >= 2016) & (df_clean["time"] <= 2022)]["value"].tolist()
#print(y_vals)
y = np.array(y_vals)
print(y)

[29.91 29.35 20.8  17.33 21.43 19.31 18.56 32.56 16.23 31.91 34.32 29.04
 26.6  18.73 22.13 15.65 29.01 15.46 15.41 20.33 16.11 29.4  28.38 20.95
 25.1  14.87 19.54 18.26 23.49 23.65 29.48 27.23 29.48 28.95 20.23 16.82
 21.12 18.31 18.09 32.1  15.64 30.75 34.09 28.93 25.61 18.16 23.39 14.54
 28.73 15.28 15.08 20.98 14.79 28.82 27.6  20.17 24.68 14.88 18.69 18.11
 22.82 23.27 28.99 27.59 29.21 28.76 19.74 16.86 21.14 17.53 18.24 31.92
 16.04 30.28 33.82 29.1  25.33 17.52 23.97 13.8  28.74 15.75 15.69 21.38
 14.22 28.38 26.53 19.47 24.03 14.89 18.63 17.82 22.4  23.42 28.53 27.07
 29.37 28.63 19.85 16.61 21.15 18.31 18.49 31.81 16.11 30.24 33.48 29.55
 25.19 16.52 25.05 13.37 29.09 16.15 16.36 21.75 14.39 28.19 27.59 20.96
 23.99 15.18 18.64 17.81 22.43 23.95 27.86 27.65 34.08 32.46 22.31 18.61
 23.94 24.25 21.51 33.44 18.94 32.1  38.   32.39 29.05 18.16 30.49 15.
 34.17 17.99 19.31 24.26 18.5  31.99 30.71 23.4  27.43 17.67 20.93 19.4
 26.38 29.78 29.47 31.43 32.83 30.12 19.95 18.81 22.42

In [ ]:
df_clean["time"] = df_clean["time"].astype(int)
df_clean = df_clean.sort_values(by=["geo", "time"]).reset_index(drop=True)


     time  value          geo  lag_1  lag_2  lag_3  lag_4  lag_5
0    2011  28.96      Austria    NaN    NaN    NaN    NaN    NaN
1    2012  29.29      Austria    NaN    NaN    NaN    NaN    NaN
2    2013  29.83      Austria    NaN    NaN    NaN    NaN    NaN
3    2014  30.00      Austria    NaN    NaN    NaN    NaN    NaN
4    2015  29.97      Austria    NaN    NaN    NaN    NaN    NaN
..    ...    ...          ...    ...    ...    ...    ...    ...
377  2018  27.07  Switzerland    NaN    NaN    NaN    NaN    NaN
378  2019  27.65  Switzerland    NaN    NaN    NaN    NaN    NaN
379  2020  31.43  Switzerland    NaN    NaN    NaN    NaN    NaN
380  2021  29.65  Switzerland    NaN    NaN    NaN    NaN    NaN
381  2022  28.38  Switzerland    NaN    NaN    NaN    NaN    NaN

[382 rows x 8 columns]


In [39]:
df_X = pd.DataFrame()
df_X["Countries"] = df_clean["geo"].unique()
df_X["lag_1"] = np.nan
df_X["lag_2"] = np.nan
df_X["lag_3"] = np.nan
df_X["lag_4"] = np.nan
df_X["lag_5"] = np.nan
for i, country in enumerate(df_X["Countries"]):
    start_year = 2011 + i
    lag_years = list(range(start_year, start_year + 5))

    country_data = df_clean[(df_clean["geo"] == country) & (df_clean["time"].astype(int).isin(lag_years))]
    country_data = country_data.sort_values("time") 

    for j, year in enumerate(lag_years):
        val_row = country_data[country_data["time"].astype(int) == year]
        if not val_row.empty:
            df_X.loc[i, f"lag_{j+1}"] = val_row.iloc[0]["value"]
            
display(df_X)


,Countries,lag_1,lag_2,lag_3,lag_4,lag_5
0,Austria,28.96,29.29,29.83,30.00,29.97
1,Belgium,29.55,29.90,29.86,29.91,29.35
2,Bulgaria,17.56,18.39,17.65,17.33,16.82
3,Croatia,21.40,21.39,21.43,21.12,21.14
4,Cyprus,19.90,19.31,18.31,17.53,18.31
5,Czechia,18.56,18.09,18.24,18.49,21.51
6,Denmark,32.10,31.92,31.81,33.44,31.01
7,Estonia,16.04,16.11,18.94,17.23,15.74
8,Finland,30.24,32.10,31.29,30.08,NaN
9,France,38.00,35.73,33.98,NaN,NaN


In [24]:
categorical_cols = ["geo"]
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df_clean[categorical_cols])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_cols))
display(one_hot_df)
print(one_hot_df.columns.tolist())

,geo_Austria,geo_Belgium,geo_Bosnia and Herzegovina,geo_Bulgaria,geo_Croatia,geo_Cyprus,geo_Czechia,geo_Denmark,geo_Estonia,geo_Finland,...,geo_Norway,geo_Poland,geo_Portugal,geo_Romania,geo_Serbia,geo_Slovakia,geo_Slovenia,geo_Spain,geo_Sweden,geo_Switzerland
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


['geo_Austria', 'geo_Belgium', 'geo_Bosnia and Herzegovina', 'geo_Bulgaria', 'geo_Croatia', 'geo_Cyprus', 'geo_Czechia', 'geo_Denmark', 'geo_Estonia', 'geo_Finland', 'geo_France', 'geo_Germany', 'geo_Greece', 'geo_Hungary', 'geo_Iceland', 'geo_Ireland', 'geo_Italy', 'geo_Latvia', 'geo_Lithuania', 'geo_Luxembourg', 'geo_Malta', 'geo_Netherlands', 'geo_Norway', 'geo_Poland', 'geo_Portugal', 'geo_Romania', 'geo_Serbia', 'geo_Slovakia', 'geo_Slovenia', 'geo_Spain', 'geo_Sweden', 'geo_Switzerland']


In [22]:
X = np.array(one_hot_df)
y = df_clean['value'].to_numpy()
print(y.shape)

(382,)


In [ ]:
XtXinv = np.linalg.inv(np.matmul(X.T, X))
m = np.matmul(XtXinv, np.matmul(X.T, y))
m